In [1]:
import numpy as np
import readgadget
import hdf5plugin
import h5py
from sys import getsizeof
import math
import pickle
import blosc

In [2]:
# input files
snapshot = '/media/fuffolo97/VolD/Riccardo/SnapshotsTry/snapdir_002/snap_002'
ptype    = [1] #[1](CDM), [2](neutrinos) or [1,2](CDM+neutrinos)

# read header
header   = readgadget.header(snapshot)
BoxSize  = header.boxsize/1e3  #Mpc/h
Nall     = header.nall         #Total number of particles
Masses   = header.massarr*1e10 #Masses of the particles in Msun/h
Omega_m  = header.omega_m      #value of Omega_m
Omega_l  = header.omega_l      #value of Omega_l
h        = header.hubble       #value of h
redshift = header.redshift     #redshift of the snapshot
Hubble   = 100.0*np.sqrt(Omega_m*(1.0+redshift)**3+Omega_l)#Value of H(z) in km/s/(Mpc/h)

# read positions, velocities and IDs of the particles
pos = readgadget.read_block(snapshot, "POS ", ptype)/1e3 #positions in Mpc/h
vel = readgadget.read_block(snapshot, "VEL ", ptype)     #peculiar velocities in km/s
ids = readgadget.read_block(snapshot, "ID  ", ptype)-1   #IDs starting from 0

In [3]:
# hf = h5py.File("/media/fuffolo97/VolD/Riccardo/SnapshotsTry/Datas/data.h5", 'w')
# hf.create_dataset("Data3", data=DATA3, compression="gzip", compression_opts=9)
# hf.close()

In [4]:
# hf = h5py.File("/media/fuffolo97/VolD/Riccardo/SnapshotsTry/Datas/data_ch2.h5", 'w')
# dset = hf.create_dataset("DataChunk", np.shape(DATA3), dtype='f', chunks = True)
# # dset.chunks
# hf.close()

# hf = h5py.File("/media/fuffolo97/VolD/Riccardo/SnapshotsTry/Datas/data_ch2.h5", 'r')
# print(hf.keys())
# a = hf.get("DataChunk")
# print(np.shape(a))
# print(a[0])
# hf.close()

In [5]:
# #data_bytes = DATA3.tobytes()
# a = blosc.compress(DATA3[0].tobytes())
# with open("/media/fuffolo97/VolD/Riccardo/SnapshotsTry/Datas/data.blosc", "wb") as f:
#     f.write(a)

In [6]:
# with open('/media/fuffolo97/VolD/Riccardo/SnapshotsTry/Datas/data.blosc', 'rb') as f:
#     dec = f.read()
#     dec = blosc.decompress(dec)

In [7]:
# import struct
# D = []
# for i in range(len(pos)):
#     D.append(struct.unpack('f', b'\xdb\x0fI@'))
# D[0]

In [8]:
from nbodykit.source.catalog import BinaryCatalog
import MAS_library as MASL

/home/fuffolo97/anaconda3/envs/nbodykit-envB/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
N_grid = 256
N_mesh = 512

with open('./data_source.dat', "wb") as ff:
    pos.tofile(ff)
    np.full(len(pos), Masses[1], dtype=np.float32).tofile(ff)
    ff.seek(0)

dtype_cust = [("Position", (np.float32, 3)),\
              ("Mass", np.float32)]

binCat = BinaryCatalog(ff.name, dtype_cust)
mesh = binCat.to_mesh(resampler='tsc', Nmesh=N_mesh,
                        compensated=True,  # deconvolving
                        interlaced=True,   # antialiasing
                        position='Position', weight="Mass",
                        BoxSize=BoxSize
                        )
delta = mesh.preview()

dens = np.zeros((N_grid, N_grid, N_grid), dtype=np.float32)
MASL.MA(pos, dens, BoxSize, 'TSC',\
        W = np.full(len(pos), Masses[1], dtype=np.float32))
dens /= np.mean(dens, dtype=np.float32)
dens -= 1.

In [10]:
mesh_2 = binCat.to_mesh(resampler='tsc', Nmesh=N_grid,
                        compensated=True,  # deconvolving
                        interlaced=True,   # antialiasing
                        position='Position', weight="Mass",
                        BoxSize=BoxSize
                        )
delta_2 = mesh_2.preview()

In [11]:
# delta_2_bis = mesh.preview(Nmesh=256)

In [12]:
mesh_3 = binCat.to_mesh(resampler='tsc', Nmesh=N_grid,
                        position='Position', weight="Mass",
                        BoxSize=BoxSize
                        )
delta_3 = mesh_3.preview().astype('float32')

In [16]:
dens_3 = delta_3
dens_3 /= np.mean(dens_3, dtype=np.float32)
dens_3 -= 1.

In [18]:
Dens_3 = np.float32(dens_3)